In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/fake-news-content-and-title'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-content-and-title/news_file_en_5.csv
/kaggle/input/fake-news-content-and-title/news_file_en_1.csv
/kaggle/input/fake-news-content-and-title/news_file_en_6.csv
/kaggle/input/fake-news-content-and-title/news_file_en_2.csv
/kaggle/input/fake-news-content-and-title/news_file_en_7.csv
/kaggle/input/fake-news-content-and-title/news_file_en_3.csv
/kaggle/input/fake-news-content-and-title/news_file_en_4.csv


In [2]:
output_dir = '/kaggle/working/'
chunk = pd.read_csv('/kaggle/input/fake-news-content-and-title/news_file_en_7.csv')

In [3]:
chunk

,id,domain,type,content,title,cleaned_content,cleaned_title,case_content,case_title
0,1958694,katehon.com,bias,Thailand decides its political future. This wi...,Thailand holds referendum on a new Constitution,thailand decide political future first mass re...,thailand hold referendum new constitution,Thailand decides political future. first mass ...,Thailand holds referendum new Constitution
1,1958717,katehon.com,bias,There are entire communities of Chechens and U...,CIA and Turks created caliphate to launch atta...,entire community chechen uighur live city town...,cia turk create caliphate launch attack russia...,entire communities Chechens Uighurs living cit...,CIA Turks created caliphate launch attacks Rus...
2,1958765,katehon.com,bias,The Russian Foreign Ministry has reported that...,Russian embassy in Damascus shelled twice today,russian foreign ministry report embassy damasc...,russian embassy damascus shell twice today,Russian Foreign Ministry reported embassy Dama...,Russian embassy Damascus shelled twice today
3,1958766,katehon.com,bias,President Bashar al-Assad performed on Monday ...,President Bashar al-Assad parktakes in Eid al-...,president bashar al assad perform monday praye...,president bashar al assad parktake eid al adha...,President Bashar al-Assad performed Monday pra...,President Bashar al-Assad parktakes Eid al-Adh...
4,1958787,katehon.com,bias,Some competitive tensions between neighboring ...,Policies and actions advocated by China and Ru...,competitive tension neighboring power russia c...,policy action advocate china russia benefit party,competitive tensions neighboring powers Russia...,Policies actions advocated China Russia benefi...
...,...,...,...,...,...,...,...,...,...
241662,2247532,lifezette.com,clickbait,Film director latest to cite intimidation\n\nA...,Scientology Strikes Again?,film director late cite intimidation alex gibn...,scientology strike,Film director latest cite intimidation Alex Gi...,Scientology Strikes ?
241663,2247533,lifezette.com,clickbait,A crowd of thousands interrupted and booed Ber...,LifeZette Sanders Delegates Boo Bernie,crowd thousand interrupt boo bernie sanders to...,lifezette sanders delegates boo bernie,crowd thousands interrupted booed Bernie Sande...,LifeZette Sanders Delegates Boo Bernie
241664,2247534,lifezette.com,clickbait,President Donald Trump kicked off his first fo...,Trump Reboot: Getting the Agenda Back on Track,president donald trump kick first foreign trip...,trump reboot get agenda back track,President Donald Trump kicked first foreign tr...,Trump Reboot : Getting Agenda Back Track
241665,2247535,lifezette.com,clickbait,President Obama wants to take away people’s gu...,All Archives,president obama want take away people gun last...,archive,President Obama wants take away peoples guns. ...,Archives


In [4]:

CONVERT = {
"rumor": "fake",
"satire": "fake",
"conspiracy": "fake",
"state": "fake",
"junksci": "fake",
"hate": "fake",
"bias": "fake",
"unreliable": "fake",
"clickbait": "fake",
"political": "reliable",
"reliable": "reliable",
"unknown": "fake"
}

In [5]:
import re
def class_convert(text, convert=CONVERT):
    pattern = re.compile('({})'.format('|'.join(convert.keys())),
                                     flags=re.IGNORECASE|re.DOTALL)
    def class_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = convert.get(match)\
                                if convert.get(match)\
                                else convert.get(match.lower())                       
        expanded_contraction = expanded_contraction
        return expanded_contraction
    
    result = pattern.sub(class_match, text)
    return result

In [6]:
import math
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    num_docs = len(corpus)
    processed_percent = 0
    percent_increment = 5
    processed_docs = 0
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # expand contractions    
        if contraction_expansion:
            doc = class_convert(doc)
        processed_docs += 1
        normalized_corpus.append(doc)
        
        # Calculate and print the progress at each percentage increment
        percent_done = math.floor((processed_docs / num_docs) * 100)
        if percent_done >= processed_percent + percent_increment:
            processed_percent = percent_done
            print(f'Processed {processed_percent}% of documents')
        
    return normalized_corpus

In [7]:
classif = normalize_corpus(chunk['type'])
chunk['type'] = classif

Processed 5% of documents
Processed 10% of documents
Processed 15% of documents
Processed 20% of documents
Processed 25% of documents
Processed 30% of documents
Processed 35% of documents
Processed 40% of documents
Processed 45% of documents
Processed 50% of documents
Processed 55% of documents
Processed 60% of documents
Processed 65% of documents
Processed 70% of documents
Processed 75% of documents
Processed 80% of documents
Processed 85% of documents
Processed 90% of documents
Processed 95% of documents
Processed 100% of documents


In [8]:
chunk

,id,domain,type,content,title,cleaned_content,cleaned_title,case_content,case_title
0,1958694,katehon.com,fake,Thailand decides its political future. This wi...,Thailand holds referendum on a new Constitution,thailand decide political future first mass re...,thailand hold referendum new constitution,Thailand decides political future. first mass ...,Thailand holds referendum new Constitution
1,1958717,katehon.com,fake,There are entire communities of Chechens and U...,CIA and Turks created caliphate to launch atta...,entire community chechen uighur live city town...,cia turk create caliphate launch attack russia...,entire communities Chechens Uighurs living cit...,CIA Turks created caliphate launch attacks Rus...
2,1958765,katehon.com,fake,The Russian Foreign Ministry has reported that...,Russian embassy in Damascus shelled twice today,russian foreign ministry report embassy damasc...,russian embassy damascus shell twice today,Russian Foreign Ministry reported embassy Dama...,Russian embassy Damascus shelled twice today
3,1958766,katehon.com,fake,President Bashar al-Assad performed on Monday ...,President Bashar al-Assad parktakes in Eid al-...,president bashar al assad perform monday praye...,president bashar al assad parktake eid al adha...,President Bashar al-Assad performed Monday pra...,President Bashar al-Assad parktakes Eid al-Adh...
4,1958787,katehon.com,fake,Some competitive tensions between neighboring ...,Policies and actions advocated by China and Ru...,competitive tension neighboring power russia c...,policy action advocate china russia benefit party,competitive tensions neighboring powers Russia...,Policies actions advocated China Russia benefi...
...,...,...,...,...,...,...,...,...,...
241662,2247532,lifezette.com,fake,Film director latest to cite intimidation\n\nA...,Scientology Strikes Again?,film director late cite intimidation alex gibn...,scientology strike,Film director latest cite intimidation Alex Gi...,Scientology Strikes ?
241663,2247533,lifezette.com,fake,A crowd of thousands interrupted and booed Ber...,LifeZette Sanders Delegates Boo Bernie,crowd thousand interrupt boo bernie sanders to...,lifezette sanders delegates boo bernie,crowd thousands interrupted booed Bernie Sande...,LifeZette Sanders Delegates Boo Bernie
241664,2247534,lifezette.com,fake,President Donald Trump kicked off his first fo...,Trump Reboot: Getting the Agenda Back on Track,president donald trump kick first foreign trip...,trump reboot get agenda back track,President Donald Trump kicked first foreign tr...,Trump Reboot : Getting Agenda Back Track
241665,2247535,lifezette.com,fake,President Obama wants to take away people’s gu...,All Archives,president obama want take away people gun last...,archive,President Obama wants take away peoples guns. ...,Archives


In [9]:
output_dir = '/kaggle/working/'
# Export cleaned text
chunk.to_csv(f'{output_dir}reclassified_7.csv', index=False)